# Metadata Archivist

**Author:** Jose Villamar

The Metadata Archivist is a lightweight file processing framework designed to parse an arbitrary number of files with heterogenous formats and transform the results into a unified output.

This tutorial is designed to guide users through the first steps needed to:
1. Implement basic parsing functions by extending the abstract parsing class of the framework.
2. Structure the output of the parsers by providing a schema template
3. Instantiate and configure the Archivist controller to coordinate input files, parsers, and resulting output

**Note:** Package requirements to run this tutorial can be found in the ```requirements.txt``` file.

In [8]:
# import the archivist framework
import metadata_archivist

# Additional packages used in this tutorial
from json import dumps
from pathlib import Path
from yaml import safe_load

## Input files

As part of the tutorial an example collection of input files is provided in the ```raw_metadata``` directory.

### config.yml
Generic configuration file containing several parameters:

In [9]:
print(Path("raw_metadata/config.yml").read_text())

parameters:
  sim_time: 100
  scale: 10
  num_procs: 2
  threads_per_proc: 8
  step_size: 0.1



### time.txt
Output of unix time wrapper command:

In [10]:
print(Path("raw_metadata/time.txt").read_text())


real	0m2.596s
user	0m15.543s
sys	0m0.632s



## File parsers

To be able to handle the input files parsers need to be defined, here we opt for one for each file.
In principle, a parser could be implemented to process both files, however for simplicity and clarity we will proceed with two.

**Note**: Although parsers need to be implemented to be able to interact with the framework, there may already exist processing frameworks for certain file types, in this case the parser implementation becomes a wrapper around these processing framework. In this example we will use [PyYAML](https://pypi.org/project/PyYAML/) to load the contents of ```config.yml```.

In [11]:
# We first extend the abstract parser class from the metadata_archivist framework
class yml_parser(metadata_archivist.AParser):
    # To instantiate the class a unique name is needed and the pattern of the file types that the parser can handle is needed.
    # This pattern is defined as a regular expression pointing to the file name.
    def __init__(self) -> None:
        super().__init__(name='yml_parser',
                         # Note that this parser will process any .yml file
                         input_file_pattern='.*\.yml',
                         # The output of the parser is described in the [JSON Schema](https://json-schema.org/overview/what-is-jsonschema) format.
                         schema={
                             'type': 'object',
                             'properties': {
                                 'parameters': {
                                     'type': 'object',
                                     'properties': {
                                         'sim_time': {
                                             'type': 'number',
                                             'description': 'total time to simulate'
                                         },
                                         'scale': {
                                             'type': 'number',
                                             'description': 'model scale'
                                         },
                                         'num_procs': {
                                             'type': 'number',
                                             'description': 'number of MPI processes'
                                         },
                                         'threads_per_proc': {
                                             'type': 'number',
                                             'description': 'number of threads used per MPI process'
                                         },
                                         'step_size': {
                                             'type': 'number',
                                             'description': 'step size for advancing simulation'
                                         }
                                     }
                                 }
                             }
                         })

    # Then the virtual parse method needs to be defined, here we use the yaml package to load the contents of the file.
    def parse(self, file_path):
        out = None
        with file_path.open() as fp:
            out = safe_load(fp)
        return out

# To help processing the time.txt file we create two functions:
def time_parser_sec(string: str) -> float:
    """"
    This functions reads a string containing time duration information in ASCII format (output from the time command) and returns its value in seconds.

    Arguments:
        string: time duration information in ASCII format (output from the time command)

    Returns:
        float value of the duration in seconds
    """
    minute_split = string.split("m")
    minutes = int(minute_split[0]) * 60 * 1000
    second_split = minute_split[1].split(".")
    seconds = int(second_split[0]) * 1000
    milis = int(second_split[1][:-1])
    return (minutes + seconds + milis) / 1000

def key_val_split(string: str, split_char: str, functor=None):
    """"
    Basic ASCII text processing function.
    Assumes input string contains a key value pair type of information, using a marker character splits the string and returns the key value pair as object.
    Optionally values can be transformed using a provided function.

    Arguments:
        string: ASCII string containing key value pair information
        split_char: ASCII character used as marker between key and value
        functor: Optional. Callable used to transform value information

    Returns:
        object containing key value pair
    """
    if functor is None:
        functor = lambda x: x
    string = string.strip()
    out = string.split(split_char)
    return {out[0].strip(): functor(out[1].strip())}


# time.txt parser
class time_parser(metadata_archivist.AParser):

    def __init__(self) -> None:
        super().__init__(name='time_parser',
                         # Note that this parser will only process files named time.txt
                         input_file_pattern='time\.txt',
                         # The output of the parser is described in the [JSON Schema](https://json-schema.org/overview/what-is-jsonschema) format.
                         schema={
                             'type': 'object',
                             'properties': {
                                 'real': {
                                     'type': 'number',
                                     'description':
                                     'the time from start to finish of the call'
                                 },
                                 'user': {
                                     'type': 'number',
                                     'description': 'amount of CPU time spent in user mode'
                                 },
                                 'sys': {
                                     'type': 'number',
                                     'description':
                                     'amount of CPU time spent in kernel mode'
                                 },
                                 'system': {
                                     '$ref': '#/properties/sys'
                                 }
                             }
                         })
    
    def parse(self, file_path) -> dict:
        out = {}
        with file_path.open("r") as fp:
            for line in fp:
                if line != '\n':
                    out.update(key_val_split(line, '\t', time_parser_sec))
        return out
        

## Structuring with a schema

To be able to structure the output of parsers a schema template describing the combined output needs to be provided.
This schema description is based on the [JSON Schema language](https://json-schema.org/learn/glossary) with additional clauses to describe structuring operations.
JSON Schema is a declarative language used to define the structure and constraints of JSON data, enabling validation and ensuring data quality. It utilizes a set of keywords to specify the properties and rules for JSON documents, allowing developers to enforce consistency and interoperability across systems.


Here we use:
* ```!parsing``` directive which introduces the usage of parsing results of a single parser.
* ```!calculate``` directive which combines through simple arithmetic the parsing results of two or more parsers.

In [12]:
# the schema template is defined as a dictionary object, naturally, it can also be stored as a JSON file.
my_schema = {
    # As the template is based on the JSON Schema language, we use the standard headers for our schema files.
    '$schema': 'https://abc',
    '$id': 'https://abc.json',
    'description': 'my example schema',
    'type': 'object',
    'properties': {
        # Here property name refers to the actual keys in the output data.
        'real_time_factor': {
            'type': 'number',
            'description': 'ratio of wall clock time to simulation time',
            # Calculate directive used to compute arithmetic expressions using parsing results of two or more parsers.
            '!calculate': {
                # The expression is defined as a string, variables can be defined inside of curly brackets.
                'expression': '{val1} / {val2}',
                # For each variable, its corresponding value from a parsing results needs to be defined.
                'variables': {
                    'val1': {
                        # The parsing directive replaces the contents of the object with the parsing results of the referenced parser.
                        '!parsing': {
                            # Here we select a subset of the available results filtered using a specific key (or set of keys).
                            'keys': ['real'],
                            # The selection result is returned as a key value pair. If only the value is needed, it can be "un-packed".
                            # Here we only need to unpack a single structure.
                            'unpack': 1
                        },
                        # Reference to the parser definition, in this framework parsers are automatically added to the definition sections of the schema.
                        # These can be referenced through the unique name provided during class instantiation.
                        '$ref': '#/$defs/time_parser'
                    },
                    'val2': {
                        '!parsing': {
                            # If the desired information is found inside a nested structure,
                            # this nesting can be indicated as a path inside the structure where each key is separated by '/' .
                            # Here subsequent keys means a deeper location inside the structure.
                            'keys': ['parameters/sim_time'],
                            # As we obtain a 2 level nested structure, the unpacking level also needs to be raised.
                            'unpack': 2
                        },
                        '$ref': '#/$defs/yml_parser'
                    }
                }
            }
        },
        'model': {
            '!parsing': {
                'keys': ['parameters/scale'],
                # Although a nested structure is selected here by parameters/scale (equivalent to parsing_results["parameters"]["scale"]).
                # We actually desire to have a scale: value key pair, hence we only use a single unpacking level.
                'unpack': 1
            },
            '$ref': '#/$defs/yml_parser'
        },
        'virtual_processes': {
            'type': 'number',
            'description': 'total number of digital processing units i.e. #MPI * #threads',
            '!calculate': {
                'expression': '{val1} * {val2}',
                'variables': {
                    'val1': {
                        '!parsing': {
                            'keys': ['parameters/num_procs'],
                            # In case we ultimately only desire terminal values from the selected structures,
                            # unpack can be set to true and the selection result will be unpacked to the highest level possible.
                            'unpack': True
                        },
                        '$ref': '#/$defs/yml_parser'
                    },
                    'val2': {
                        '!parsing': {
                            'keys': ['parameters/threads_per_proc'],
                            'unpack': True
                        },
                        '$ref': '#/$defs/yml_parser'
                    }
                }
            }
        }
    }
}

## Getting all together

To be able to handle an arbitrary number of parsers and interpret the schema, the Metadata Archivist framework employs a ```Formatter``` class which handles parsing execution and structuring of its output.
Together with the ```Archivist``` class which handles directory or archive exploration and exporting the structured output, the framework is capable of handling an arbitrary number of files and formats resulting in an unified structured output.

In [13]:
# Formatter instantiation
my_parser = metadata_archivist.Formatter(parsers=[time_parser(), yml_parser()], schema=my_schema, lazy_load=True)

arch = metadata_archivist.Archivist(path=Path('raw_metadata'),
                                    formatter=my_parser,
                                    output_directory="./",
                                    output_file="metadata.json",
                                    overwrite=True,
                                    auto_cleanup=True,
                                    verbose='info',
                                    add_description=True,
                                    add_type=True)

arch.parse()

print("\nResulting schema:")
print(dumps(my_parser.schema, indent=4))

print("\nResulting metadata:")
print(dumps(arch.get_metadata(), indent=4))


No argument found for: 'extraction_directory' initializing by default: '.'


Extracting:
        Output path: .
        Extraction path: .
        Remove extracted: True
Unpacking archive...
Exploration of directory: raw_metadata
    processing file: time.txt
    processing file: config.yml
Done!
parsing files ...
Done!
Compiling metadata...
Done!
Cleaning meta file: raw_metadata/time.txt.meta
Cleaning meta file: raw_metadata/config.yml.meta
Done!



Resulting schema:
{
    "$schema": "https://abc",
    "$id": "https://abc.json",
    "description": "my example schema",
    "type": "object",
    "properties": {
        "real_time_factor": {
            "type": "number",
            "description": "ratio of wall clock time to simulation time",
            "!calculate": {
                "expression": "{val1} / {val2}",
                "variables": {
                    "val1": {
                        "!parsing": {
                            "keys": [
                                "real"
                            ],
                            "unpack": 1
                        },
                        "$ref": "#/$defs/time_parser"
                    },
                    "val2": {
                        "!parsing": {
                            "keys": [
                                "parameters/sim_time"
                            ],
                            "unpack": 2
                        },
                